In [1]:
import re
import time
import random
import numpy as np
import pandas as pd 
from copy import deepcopy
from threading import Thread
from datetime import datetime

In [2]:
class CPU:
    def __init__(self, number_of_cores=0, clock_rate_in_hz=0.0, family_name='', denomination=''):
        """
        Initializes a new CPU with the given parameters.
        """
        self.number_of_cores = number_of_cores
        self.clock_rate_in_hz = clock_rate_in_hz
        self.family_name = family_name
        self.denomination = denomination

    def duplicate(self):
        """
        Creates and returns a deep copy of this CPU.
        """
        return deepcopy(self)

    def outperforms(self, cpu_to_compare_with, multithreaded_program_execution):
        """
        Returns True if this CPU outperforms the one passed in as a parameter for the specified type of program.
        """

        # Priority mapping for CPU families and denominations
        cpu_hierarchy = {
            'core i9': 4,
            'core i7': 3,
            'core i5': 2,
            'core i3': 1
        }

        # Determine the CPU type
        this_cpu_type = f"{self.family_name} {self.denomination}".lower()
        other_cpu_type = f"{cpu_to_compare_with.family_name} {cpu_to_compare_with.denomination}".lower()

        # Compare based on family/denomination hierarchy
        this_priority = cpu_hierarchy.get(this_cpu_type, 0)
        other_priority = cpu_hierarchy.get(other_cpu_type, 0)

        if this_priority != other_priority:
            return this_priority > other_priority

        # If the same family, compare based on multithread or single-thread performance
        if multithreaded_program_execution:
            return self.number_of_cores > cpu_to_compare_with.number_of_cores
        else:
            return self.clock_rate_in_hz > cpu_to_compare_with.clock_rate_in_hz

    def __eq__(self, other):
        """
        Compares if two CPU objects are equivalent in terms of performance.
        This allows using '==' for CPU objects.
        """
        if not isinstance(other, CPU):
            return NotImplemented
        return (self.family_name == other.family_name and
                self.denomination == other.denomination and
                self.number_of_cores == other.number_of_cores and
                self.clock_rate_in_hz == other.clock_rate_in_hz)


In [3]:
class Worker:
    def __init__(self, ID=0, cpu_info=None, current_global_cpu_time=0.0, available_memory_size=0.0, 
                 available_disk_size=0.0, assigned_jobs=None, connection_bandwidth_with_master_pc=0.0, 
                 connection_delay_with_master_pc=0.0, cpu_usage_in_percentage=0.0, name="", 
                 original_available_memory_size=0.0, original_available_disk_size=0.0, 
                 original_connection_bandwidth_with_master_pc=0.0, original_connection_delay_with_master_pc=0.0):
        self.ID = ID
        self.cpu_info = cpu_info
        self.available_memory_size = available_memory_size
        self.available_disk_size = available_disk_size
        self.connection_bandwidth_with_master_pc = connection_bandwidth_with_master_pc
        self.connection_delay_with_master_pc = connection_delay_with_master_pc
        self.original_available_memory_size = original_available_memory_size
        self.original_available_disk_size = original_available_disk_size
        self.original_connection_bandwidth_with_master_pc = original_connection_bandwidth_with_master_pc
        self.original_connection_delay_with_master_pc = original_connection_delay_with_master_pc
        self.assigned_jobs = assigned_jobs if assigned_jobs is not None else []
        self.cpu_usage_in_percentage = cpu_usage_in_percentage
        self.name = name
        self.current_global_cpu_time = current_global_cpu_time
        self.base10_name = 0
    def duplicate(self):
        return deepcopy(self)

    def print(self):
        print("********** Worker PC Details **********")
        print(f"ID: {self.ID}")
        print(f"Name: {self.name}")
        print(f"Current Global CPU Time: {self.current_global_cpu_time}")
        print(f"CPU Number Of Cores: {self.cpu_info.number_of_cores}")
        print(f"CPU Clock Rate In GHz: {self.cpu_info.clock_rate_in_hz / 1_000_000_000}")
        print(f"CPU Name: {self.cpu_info.family_name} {self.cpu_info.denomination}")
        print(f"Available Memory Size: {self.available_memory_size}")
        print(f"Available Disk Size: {self.available_disk_size}")
        print(f"Connection Bandwidth With Master PC: {self.connection_bandwidth_with_master_pc}")
        print(f"Connection Delay With Master PC: {self.connection_delay_with_master_pc}")
        print(f"CPU Usage In Percentage: {self.cpu_usage_in_percentage}")
        print("----- Assigned Jobs -----")
        for job in self.assigned_jobs:
            print(f"Job Name: {job.name}")
        print("********** End **********\n\n")

    def can_handle_job(self, job):
        if self.available_memory_size < job.required_memory_size_for_execution:
            return False
        if self.available_disk_size < job.required_disk_size_for_execution:
            return False
        if self.cpu_info.number_of_cores < job.thread_process_count:
            return False
        return True

    def reset_resource_usage(self, job):
        self.cpu_usage_in_percentage = 0.0
        self.available_disk_size += job.required_disk_size_for_execution
        self.available_memory_size += job.required_memory_size_for_execution

    def execute_job(self, job):
        job_execution_thread = Thread(target=job.run)
        job_execution_thread.start()


In [4]:
class Job(Thread):  # Inherit from threading.Thread to make the class runnable
    def __init__(self, ID=0, standard_processing_durations=None, required_memory_size_for_execution=0.0, 
                 required_disk_size_for_execution=0.0, assigned_worker=None, job_current_cpu_time=0.0, 
                 docker_file_size=0.0, assignment_time=0, estimated_result_file_size=0.0, 
                 docker_file_generation_duration_on_master_pc=0.0, currently_being_processed_on_assigned_worker=False, 
                 thread_process_count=1, currently_assigned_to_worker=False, 
                 finished_being_processed_on_assigned_worker=False, name="", induced_cpu_usage_increase_percentage=0.0):
        super().__init__()
        self.ID = ID
        self.standard_processing_durations = standard_processing_durations or {}
        self.required_memory_size_for_execution = required_memory_size_for_execution
        self.required_disk_size_for_execution = required_disk_size_for_execution
        self.assigned_worker = assigned_worker
        self.job_current_cpu_time = job_current_cpu_time
        self.docker_file_size = docker_file_size
        self.assignment_time = assignment_time
        self.estimated_result_file_size = estimated_result_file_size
        self.docker_file_generation_duration_on_master_pc = docker_file_generation_duration_on_master_pc
        self.currently_being_processed_on_assigned_worker = currently_being_processed_on_assigned_worker
        self.thread_process_count = thread_process_count
        self.currently_assigned_to_worker = currently_assigned_to_worker
        self.finished_being_processed_on_assigned_worker = finished_being_processed_on_assigned_worker
        self.name = name
        self.induced_cpu_usage_increase_percentage = induced_cpu_usage_increase_percentage

    def duplicate(self, job):
        return deepcopy(self)

    def run(self):
        try:
            self.currently_being_processed_on_assigned_worker = True
            if self.assigned_worker:
                print(f"{self.assigned_worker.name}---<< {self.name} <<---")

                # Simulating processing time based on the assigned worker's CPU info
                family_name = self.assigned_worker.cpu_info['family_name']
                denomination = self.assigned_worker.cpu_info['denomination']
                cores = self.assigned_worker.cpu_info['number_of_cores']
                key = f"{family_name}-{denomination}-{cores}"

                # Sleep for the standard processing duration
                if key in self.standard_processing_durations:
                    processing_time = self.standard_processing_durations[key]
                    time.sleep(processing_time)

                # Processing complete
                self.finished_being_processed_on_assigned_worker = True
                self.currently_being_processed_on_assigned_worker = False
                print(f"{self.assigned_worker.name}--->> {self.name} >>---")

                # Update worker's resources after processing
                self.assigned_worker.assigned_jobs.remove(self)
                self.assigned_worker.available_disk_size += self.required_disk_size_for_execution
                self.assigned_worker.available_memory_size += self.required_memory_size_for_execution
                self.assigned_worker.cpu_usage_in_percentage -= self.induced_cpu_usage_increase_percentage

                # Reset job's worker assignment
                self.currently_assigned_to_worker = False
                self.assigned_worker = None

        except Exception as e:
            print(f"Job execution failed: {e}")

    def print(self):
        print("++++++++++ Job Details ++++++++++")
        print(f"ID: {self.ID}")
        print(f"Name: {self.name}")
        for cpu, duration in self.standard_processing_durations.items():
            print(f"Standard Processing Duration On {cpu}: {duration}")
        print(f"Required Memory Size For Execution: {self.required_memory_size_for_execution}")
        print(f"Required Disk Size For Execution: {self.required_disk_size_for_execution}")
        print(f"Docker File Size: {self.docker_file_size}")
        print(f"Arrival Time: {self.assignment_time}")
        print(f"Thread Process Count: {self.thread_process_count}")
        print(f"Estimated Result File Size: {self.estimated_result_file_size}")
        print(f"Docker File Generation Duration On Master PC: {self.docker_file_generation_duration_on_master_pc}")
        print(f"Current CPU Time: {self.job_current_cpu_time}")
        print(f"Currently Assigned To Worker: {self.currently_assigned_to_worker}")
        print(f"Currently Being Processed On Assigned Worker: {self.currently_being_processed_on_assigned_worker}")
        print(f"Finished Being Processed On Assigned Worker: {self.finished_being_processed_on_assigned_worker}")
        if self.assigned_worker:
            print(f"Assigned Worker: {self.assigned_worker.name}")
        print("++++++++++ End ++++++++++\n\n")


In [5]:
class Match:
    def __init__(self, value, pheromone=1):
        self.value = value  # value is expected to be a tuple (job, worker)
        self.pheromone = pheromone
        self.processing_duration = self.calculate_processing_duration()

    def calculate_processing_duration(self):
        job, worker = self.value
        # Use the worker's name (e.g., 'PC1') to get the correct duration from the job's dictionary
        return job.standard_processing_durations[worker.name]  # Assume worker has an 'name' or identifier like 'PC1'


In [6]:
class Ant:
    def __init__(self, id, path=[]):
        self.id = id
        self.path = path

In [7]:
def generate_paths(ants, jobs, matches, alpha, beta):
    for ant in ants:
        ant.path = []
        for job in jobs:
            matching_elements = [match for match in matches if match.value[0] == job]
            p = calculate_probabilities(matching_elements, alpha, beta)  # calculate probabilities (worker, probability)
            r = random.random()
            w = pick_worker(r, p)
            ant.path.append((job, w))  # append job-worker pair to the ant's path
    return ants

def calculate_probabilities(matches, alpha, beta):
    """
    Calculate the probability of selecting each worker for the given job.
    
    :param matches: List of Match objects where value[0] is the job and value[1] is the worker.
    :param alpha: Weight for pheromone importance.
    :param beta: Weight for processing duration (distance) importance.
    :return: List of tuples (worker, probability).
    """
    desirabilities = []
    total_desirability = 0

    # Calculate desirability for each match and sum them for normalization
    for match in matches:
        worker = match.value[1]  # worker is the second item in the tuple (job, worker)
        pheromone = match.pheromone
        processing_duration = match.processing_duration
        proximity = 1 / processing_duration  # Inverse of the distance is the proximity

        # Desirability formula: τ_ik^α * η_ik^β
        desirability = (pheromone ** alpha) * (proximity ** beta)
        desirabilities.append((worker, desirability))
        total_desirability += desirability

    # Normalize desirabilities to get probabilities
    probabilities = [(worker, desirability / total_desirability) for worker, desirability in desirabilities]

    return probabilities

def pick_worker(random_value, worker_probabilities):
    """
    Select a worker based on the calculated probabilities and a random value.
    
    :param random_value: Random number between 0 and 1.
    :param worker_probabilities: List of tuples (worker, probability).
    :return: The selected worker.
    """
    cumulative_prob = 0

    for worker, probability in worker_probabilities:
        cumulative_prob += probability
        if random_value <= cumulative_prob:
            return worker  # Return the exact worker when the random value falls within the cumulative probability

    return worker_probabilities[-1][0]  # Fallback to return the last worker in case of rounding issues


In [8]:
def pheromone_update(ants, matches, evap_coeff, Q):
    """
    Updates the pheromone levels on the matches based on the ants' paths.

    :param ants: List of ants, where each ant has a path attribute (a list of job-worker tuples).
    :param matches: List of Match objects with pheromone levels to be updated.
    :param evap_coeff: Coefficient for pheromone evaporation (ρ).
    :param Q: Constant value for pheromone deposition.
    :return: Updated matches with new pheromone levels.
    """
    # Evaporate pheromones on all matches
    for match in matches:
        match.pheromone *= evap_coeff  # Apply evaporation to all pheromone values

    # Update pheromones based on each ant's path
    for ant in ants:
        l = max_worker_processing_duration(ant.path)  # Max processing duration across workers
        pheromone = Q / l      # Calculate pheromone to deposit (Δτ_ij)

        for match in matches:
            if match.value in ant.path:  # Check if the job-worker pair was part of the ant's path
                match.pheromone += pheromone  # Add pheromone based on the ant's contribution

    return matches

def max_worker_processing_duration(path):
    """
    Calculates the maximum processing duration for any worker in the ant's path, 
    considering the worker's ability to run jobs simultaneously based on resource availability.
    
    :param path: A list of tuples (job, worker) representing the path of an ant.
    :return: The maximum processing duration across all workers.
    """
    # Dictionary to store jobs per worker, using worker.name as the key
    worker_jobs = {}

    # Group jobs by worker
    for job, worker in path:
        if worker.name not in worker_jobs:
            worker_jobs[worker.name] = {'worker': worker, 'jobs': []}  # Store worker object alongside jobs
        worker_jobs[worker.name]['jobs'].append(job)

    max_duration = 0

    # Evaluate the total processing duration per worker
    for worker_info in worker_jobs.values():
        worker = worker_info['worker']  # Get the worker object
        jobs = worker_info['jobs']  # Get the list of jobs
        worker_duration = calculate_worker_duration(jobs, worker)
        max_duration = max(max_duration, worker_duration)

    return max_duration

def calculate_worker_duration(jobs, worker):
    """
    Calculates the total duration for a worker considering simultaneous job execution
    based on the worker's available resources (memory, disk, CPU cores).
    
    :param jobs: List of jobs assigned to the worker.
    :param worker: The worker object with available resources (memory, disk, cores).
    :return: The total time required for the worker to complete all jobs.
    """
    # Sort jobs by their processing duration as a heuristic to allocate resources efficiently
    jobs = sorted(jobs, key=lambda job: job.standard_processing_durations[worker.name])

    total_time = 0
    # Initialize available resources for the worker
    available_memory = worker.available_memory_size
    available_disk = worker.available_disk_size
    available_cores = worker.cpu_info.number_of_cores

    # List of jobs that are executing in parallel with their remaining durations
    executing_jobs = []

    # List to track remaining job processing times
    remaining_durations = {job: job.standard_processing_durations[worker.name] for job in jobs}

    while jobs or executing_jobs:
        # Try to allocate more jobs to be executed in parallel
        remaining_jobs = []
        for job in jobs:
            # Check if the worker can handle this job along with the currently executing jobs
            if (available_memory >= job.required_memory_size_for_execution and
                available_disk >= job.required_disk_size_for_execution and
                available_cores >= job.thread_process_count):
                
                # If the worker can handle this job, add it to the executing jobs
                executing_jobs.append(job)
                available_memory -= job.required_memory_size_for_execution
                available_disk -= job.required_disk_size_for_execution
                available_cores -= job.thread_process_count
            else:
                # If the worker can't handle it, put it back in the remaining jobs
                remaining_jobs.append(job)

        # If no jobs are currently executing, we advance time by the duration of the job that finishes next
        if executing_jobs:
            # Find the job with the shortest remaining processing time in the current executing jobs
            min_duration = min(remaining_durations[job] for job in executing_jobs)
            total_time += min_duration

            # Remove completed jobs and update available resources
            completed_jobs = []
            for job in executing_jobs:
                remaining_durations[job] -= min_duration  # Reduce the remaining duration for this job
                if remaining_durations[job] == 0:  # Job has finished
                    available_memory += job.required_memory_size_for_execution
                    available_disk += job.required_disk_size_for_execution
                    available_cores += job.thread_process_count
                    completed_jobs.append(job)

            # Remove completed jobs from the executing list
            executing_jobs = [job for job in executing_jobs if job not in completed_jobs]

        # Update the jobs list with the remaining jobs for the next iteration
        jobs = remaining_jobs

    return total_time

In [22]:
def evaluate(jobs, matches):
    """
    Evaluates the matches to determine the optimal path with the minimum processing duration.
    
    :param jobs: List of jobs.
    :param matches: List of Match objects (job-worker pairs with pheromone and processing duration).
    :return: A tuple containing the optimal path and the total processing duration.
             The optimal path is a list of tuples (job, worker), and the total duration is an integer.
    """
    optimal_path = []
    worker_jobs = {}  # Dictionary to store jobs assigned to each worker

    # Build the optimal path by selecting the match with the highest pheromone level for each job
    for job in jobs:
        # Filter matches that correspond to the current job
        job_matches = [match for match in matches if match.value[0] == job]

        # Select the match with the highest pheromone level (most likely assignment)
        best_match = max(job_matches, key=lambda m: m.pheromone)

        # Add the job-worker pair to the optimal path
        worker = best_match.value[1]
        optimal_path.append((job, worker))

        # Store the job under the worker's list of jobs
        if worker not in worker_jobs:
            worker_jobs[worker] = []
        worker_jobs[worker].append(job)

    # Calculate the total processing duration based on the max worker processing duration
    total_processing_duration = 0
    for worker, assigned_jobs in worker_jobs.items():
        worker_duration = calculate_worker_duration(assigned_jobs, worker)
        total_processing_duration = max(total_processing_duration, worker_duration)

    return optimal_path, total_processing_duration
    
def format_duration(seconds):
    """
    Converts a duration from seconds to a string in the format of hours, minutes, and seconds.
    
    :param seconds: The total duration in seconds.
    :return: A string representing the duration in 'H hours M minutes S seconds' format.
    """
    hours, remainder = divmod(seconds, 3600)  # Get hours and the remainder seconds
    minutes, seconds = divmod(remainder, 60)  # Get minutes and remaining seconds

    # Build the formatted string
    duration_str = f"{int(hours)} hours {int(minutes)} minutes {int(seconds)} seconds"
    
    return duration_str


In [10]:
def ACO(jobs, matches, ants, alpha, beta, evap_coeff, Q, max_iterations=100, tolerance=1e-5, patience=50):
    """
    Runs the Ant Colony Optimization (ACO) algorithm to find the optimal job-worker assignments.
    
    :param jobs: List of jobs.
    :param matches: List of Match objects (job-worker pairs with pheromone and processing duration).
    :param ants: List of Ant objects, where each ant has a path.
    :param alpha: Pheromone influence factor.
    :param beta: Heuristic information influence factor.
    :param evap_coeff: Coefficient for pheromone evaporation (ρ).
    :param Q: Constant value for pheromone deposition.
    :param max_iterations: Maximum number of iterations to run.
    :param tolerance: Threshold to stop the algorithm when the change in total duration is small enough.
    :param patience: Number of consecutive iterations without significant improvement before stopping.
    :return: A tuple containing the optimal path and the total processing duration.
    """
    previous_duration = float('inf')  # Start with an infinitely large duration
    iteration = 0
    no_improvement_count = 0  # Counter for iterations without significant improvement

    while iteration < max_iterations:
        # Generate paths for ants based on current pheromone levels
        ants = generate_paths(ants, jobs, matches, alpha, beta)

        # Update pheromone levels based on the paths taken by the ants
        matches = pheromone_update(ants, matches, evap_coeff, Q)

        # Evaluate the current best path and its total duration
        optimal_path, total_duration = evaluate(jobs, matches)

        # Calculate the difference in total duration between iterations
        duration_difference = abs(previous_duration - total_duration)

        # Check if the difference is below the tolerance (convergence criteria)
        if duration_difference < tolerance:
            no_improvement_count += 1
        else:
            no_improvement_count = 0  # Reset the count if there's significant improvement

        # If there has been no significant improvement for a certain number of iterations, break
        if no_improvement_count >= patience:
            print(f"No significant improvement for {patience} consecutive iterations. Stopping.")
            print(f"Converged at iteration {iteration} with duration difference {duration_difference}")
            break

        # If the difference is below the tolerance and we have met the patience criteria, stop
        # if no_improvement_count == 0 and duration_difference < tolerance:
        #     print(f"Converged at iteration {iteration} with duration difference {duration_difference}")
        #     break

        # Update previous duration and increment iteration counter
        previous_duration = total_duration
        iteration += 1

    # Return the best path found and its total processing duration
    return optimal_path, total_duration


In [36]:
jobData = pd.read_csv("data/jobs3.csv")
workersData = pd.read_csv("data/workers.csv")

In [37]:
jobData.head()

,Job Name,Standard Processing Duration PC1,Standard Processing Duration PC2,Standard Processing Duration PC3,Standard Processing Duration PC4,Standard Processing Duration PC5,Standard Processing Duration PC6,Required Memory Size For Execution,Required Disk Size For Execution,Docker File Size,Docker File Generation Duration On Master PC,Estimated Result File Size,Thread Process Count
0,Network Simulator,02:14:46,01:06:44,00:54:21,00:39:40,00:36:09,01:05:59,0.0272GB,0.381GB,0.392GB,00:01:26,17KB,1
1,Optimization Algorithm,00:41:43,00:26:38,00:20:27,00:13:53,00:13:10,00:25:24,0.0276GB,1.37GB,1.5GB,00:02:21,8KB,1
2,DCGAN,01:37:14,01:09:28,00:22:44,00:12:45,00:11:15,01:09:43,1.5476GB,1.87GB,1.9GB,00:04:15,20KB,17
3,RNN,00:17:43,00:12:01,00:07:03,00:05:37,00:04:49,00:12:15,1.2144GB,1.83GB,1.9GB,00:03:36,8KB,17
4,CNN,00:26:04,00:22:22,00:07:07,00:05:24,00:04:47,00:21:18,1.4012GB,1.83GB,1.9GB,00:06:19,4KB,17


In [38]:
def create_jobs_from_df(df):
    jobs = []
    
    for index, row in df.iterrows():
        # Parse the processing durations for each PC
        standard_processing_durations = {
            'PC1': parse_time_to_seconds(row['Standard Processing Duration PC1']),
            'PC2': parse_time_to_seconds(row['Standard Processing Duration PC2']),
            'PC3': parse_time_to_seconds(row['Standard Processing Duration PC3']),
            'PC4': parse_time_to_seconds(row['Standard Processing Duration PC4']),
            'PC5': parse_time_to_seconds(row['Standard Processing Duration PC5']),
            'PC6': parse_time_to_seconds(row['Standard Processing Duration PC6']),
        }

        # Convert memory, disk sizes, and other attributes
        required_memory_size = parse_size(row['Required Memory Size For Execution'])
        required_disk_size = parse_size(row['Required Disk Size For Execution'])
        docker_file_size = parse_size(row['Docker File Size'])
        estimated_result_file_size = parse_size(row['Estimated Result File Size'])
        docker_file_gen_duration = parse_time_to_seconds(row['Docker File Generation Duration On Master PC'])

        # Create a Job object with the parsed values
        job = Job(
            ID = index,
            name=row['Job Name'],
            standard_processing_durations=standard_processing_durations,
            required_memory_size_for_execution=required_memory_size,
            required_disk_size_for_execution=required_disk_size,
            docker_file_size=docker_file_size,
            estimated_result_file_size=estimated_result_file_size,
            docker_file_generation_duration_on_master_pc=docker_file_gen_duration,
            thread_process_count=int(row['Thread Process Count'])
        )
        
        jobs.append(job)
    
    return jobs

# Helper function to parse sizes (e.g., GB, MB, KB) to bytes
def parse_size(size_str):
    size_str = size_str.lower()
    if 'gb' in size_str:
        return float(size_str.split('gb')[0].strip()) * 1024 * 1024 * 1024
    elif 'mb' in size_str:
        return float(size_str.split('mb')[0].strip()) * 1024 * 1024
    elif 'kb' in size_str:
        return float(size_str.split('kb')[0].strip()) * 1024
    elif 'b' in size_str:
        return float(size_str.split('b')[0].strip())
    return 0.0

# Helper function to parse time strings to seconds (e.g., HH:mm:ss)
def parse_time_to_seconds(time_str):
    time_parts = list(map(int, time_str.split(':')))
    return time_parts[0] * 3600 + time_parts[1] * 60 + time_parts[2]

In [39]:
jobs = create_jobs_from_df(jobData)

# Printing job details for verification
for job in jobs:
    job.print()

++++++++++ Job Details ++++++++++
ID: 0
Name: Network Simulator
Standard Processing Duration On PC1: 8086
Standard Processing Duration On PC2: 4004
Standard Processing Duration On PC3: 3261
Standard Processing Duration On PC4: 2380
Standard Processing Duration On PC5: 2169
Standard Processing Duration On PC6: 3959
Required Memory Size For Execution: 29205777.6128
Required Disk Size For Execution: 409095634.944
Docker File Size: 420906795.008
Arrival Time: 0
Thread Process Count: 1
Estimated Result File Size: 17408.0
Docker File Generation Duration On Master PC: 86
Current CPU Time: 0.0
Currently Assigned To Worker: False
Currently Being Processed On Assigned Worker: False
Finished Being Processed On Assigned Worker: False
++++++++++ End ++++++++++


++++++++++ Job Details ++++++++++
ID: 1
Name: Optimization Algorithm
Standard Processing Duration On PC1: 2503
Standard Processing Duration On PC2: 1598
Standard Processing Duration On PC3: 1227
Standard Processing Duration On PC4: 833
Stan

In [40]:
def create_workers_from_df(df):
    # List to store Worker objects
    workers = []
    
    for index, row in df.iterrows():
        # Create a new CPU object
        cpu_info = CPU(
            number_of_cores=int(row['Available CPU Core Number']),
            clock_rate_in_hz=convert_to_hz(row['CPU Clock Rate']),
            family_name=row['CPU Family Name'].strip(),
            denomination=row['CPU Denomination'].strip()
        )
    
        # Create a new Worker object
        worker = Worker(
            ID=index,
            cpu_info=cpu_info,
            available_memory_size=convert_to_bytes(row['Available Memory Size']),
            available_disk_size=convert_to_bytes(row['Available Disk Size']),
            connection_bandwidth_with_master_pc=convert_to_bytes(row['Connection Bandwidth With Master PC']),
            connection_delay_with_master_pc=convert_to_seconds(row['Connection Delay With Master PC']),
            name=row['Worker PC Name'].strip(),
            cpu_usage_in_percentage=0.0,
            current_global_cpu_time=0.0
        )
    
        # Add the worker to the list
        workers.append(worker)
        
    return workers
    
# Function to convert CPU clock rate from various formats to Hz
def convert_to_hz(value):
    value = value.lower().strip()
    if 'ghz' in value:
        return float(re.split(r"ghz", value)[0].strip()) * 1_000_000_000
    elif 'mhz' in value:
        return float(re.split(r"mhz", value)[0].strip()) * 1_000_000
    elif 'khz' in value:
        return float(re.split(r"khz", value)[0].strip()) * 1_000
    elif 'hz' in value:
        return float(re.split(r"hz", value)[0].strip())
    return float(value)

# Function to convert memory and disk size to bytes
def convert_to_bytes(value):
    value = value.lower().strip()
    if 'gb' in value:
        return float(re.split(r"gb", value)[0].strip()) * 1_073_741_824
    elif 'mb' in value:
        return float(re.split(r"mb", value)[0].strip()) * 1_048_576
    elif 'kb' in value:
        return float(re.split(r"kb", value)[0].strip()) * 1_024
    elif 'b' in value:
        return float(re.split(r"b", value)[0].strip())
    return float(value)

# Function to parse connection delay from time format to seconds
def convert_to_seconds(value):
    try:
        time_obj = datetime.strptime(value.strip(), '%H:%M:%S')
        return time_obj.hour * 3600 + time_obj.minute * 60 + time_obj.second
    except ValueError:
        return float(value)


In [41]:
workers = create_workers_from_df(workersData)

# Print the details of each worker
for worker in workers:
    worker.print()

********** Worker PC Details **********
ID: 0
Name: PC1
Current Global CPU Time: 0.0
CPU Number Of Cores: 4
CPU Clock Rate In GHz: 1.7
CPU Name: core i3
Available Memory Size: 2147483648.0
Available Disk Size: 68719476736.0
Connection Bandwidth With Master PC: 104857600.0
Connection Delay With Master PC: 0
CPU Usage In Percentage: 0.0
----- Assigned Jobs -----
********** End **********


********** Worker PC Details **********
ID: 1
Name: PC2
Current Global CPU Time: 0.0
CPU Number Of Cores: 4
CPU Clock Rate In GHz: 2.6
CPU Name: core i5
Available Memory Size: 2147483648.0
Available Disk Size: 68719476736.0
Connection Bandwidth With Master PC: 104857600.0
Connection Delay With Master PC: 0
CPU Usage In Percentage: 0.0
----- Assigned Jobs -----
********** End **********


********** Worker PC Details **********
ID: 2
Name: PC3
Current Global CPU Time: 0.0
CPU Number Of Cores: 8
CPU Clock Rate In GHz: 3.4
CPU Name: core i7
Available Memory Size: 4294967296.0
Available Disk Size: 68719476

In [42]:
def create_matches(jobs,workers,VERBOSE = 0):
    #list to create matches
    matches = []
    
    for job in jobs: 
        for worker in workers:
            # Check if the worker meets the conditions to handle the job
            if worker.can_handle_job(job):
                match = Match(value=(job,worker))
                matches.append(match)
                if VERBOSE:
                    print(f"{job.name} {worker.name}")
    return matches
matches = create_matches(jobs,workers)    

In [43]:
ants = []
for i in range(1, len(jobs)+1):
    ants.append(Ant(id=i))

In [44]:
optimal_path, total_duration = ACO(jobs, matches, ants, alpha=1, beta=1, evap_coeff=0.9, Q=100, max_iterations=1000, tolerance=1e-5)

No significant improvement for 50 consecutive iterations. Stopping.
Converged at iteration 51 with duration difference 0


In [45]:
def display_duration_per_worker(optimal_path):
	worker_jobs = {}
	
	for job, worker in optimal_path: 
		if worker not in worker_jobs:
			worker_jobs[worker] = []
		worker_jobs[worker].append(job) 
	for worker, assigned_jobs in worker_jobs.items():
		worker_duration = calculate_worker_duration(assigned_jobs, worker)
		print(f"Worker: {worker.name}, Total Duration: {format_duration(worker_duration)}")

# Example usage after running ACO and getting optimal_path
display_duration_per_worker(optimal_path)

Worker: PC5, Total Duration: 1 hours 36 minutes 25 seconds
Worker: PC4, Total Duration: 0 hours 13 minutes 53 seconds


In [46]:
print("Optimal Path:")
for job, worker in optimal_path:
    print(f"Job: {job.name}, Worker: {worker.name}")
print(f"Total Processing Duration: {format_duration(total_duration)}")

Optimal Path:
Job: Network Simulator, Worker: PC5
Job: Optimization Algorithm, Worker: PC5
Job: DCGAN, Worker: PC5
Job: RNN, Worker: PC5
Job: CNN, Worker: PC5
Job: FFmpeg, Worker: PC5
Job: Converter, Worker: PC5
Job: Palabos, Worker: PC5
Job: Flow, Worker: PC4
Job: Network Simulator, Worker: PC5
Job: Optimization Algorithm, Worker: PC5
Job: DCGAN, Worker: PC5
Job: RNN, Worker: PC5
Job: CNN, Worker: PC5
Job: FFmpeg, Worker: PC5
Job: Converter, Worker: PC5
Job: Palabos, Worker: PC5
Job: Flow, Worker: PC5
Job: Network Simulator, Worker: PC5
Job: Optimization Algorithm, Worker: PC4
Job: DCGAN, Worker: PC5
Job: RNN, Worker: PC5
Job: CNN, Worker: PC5
Job: FFmpeg, Worker: PC5
Job: Converter, Worker: PC5
Job: Palabos, Worker: PC5
Job: Flow, Worker: PC5
Total Processing Duration: 1 hours 36 minutes 25 seconds
